In [25]:
import pandas as pd
import numpy as np
from datetime import datetime
import ast
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
def load_and_process_trades():
    """Load and process trade history from CSV"""
    try:
        df = pd.read_csv('trade_history.csv')
        df = df.dropna(subset=['Trade_History'])
        
        processed_data = []
        for _, row in df.iterrows():
            port_id = row['Port_IDs']
            trades = ast.literal_eval(row['Trade_History']) if isinstance(row['Trade_History'], str) else []
            
            if trades:
                for trade in trades:
                    trade['Port_ID'] = port_id
                    # Classify position type
                    trade['position_type'] = f"{trade['side'].lower()}_{trade['positionSide'].lower()}"
                    processed_data.append(trade)
        
        trades_df = pd.DataFrame(processed_data)
        trades_df['time'] = pd.to_datetime(trades_df['time'], unit='ms')
        
        print(f"Processed {len(trades_df)} trades from {len(df)} portfolios")
        return trades_df
    
    except Exception as e:
        print(f"Error processing trades: {str(e)}")
        raise

In [27]:

def calculate_portfolio_metrics(trades_df):
    """Calculate comprehensive metrics for each portfolio"""
    metrics = []
    
    for port_id in trades_df['Port_ID'].unique():
        port_trades = trades_df[trades_df['Port_ID'] == port_id]
        
        # Position Analysis
        total_positions = len(port_trades)
        win_positions = len(port_trades[port_trades['realizedProfit'] > 0])
        loss_positions = len(port_trades[port_trades['realizedProfit'] < 0])
        win_rate = win_positions / total_positions if total_positions > 0 else 0
        
        # Position Types Analysis
        position_types = port_trades['position_type'].value_counts().to_dict()
        
        # PnL Analysis
        total_pnl = port_trades['realizedProfit'].sum()
        avg_win = port_trades[port_trades['realizedProfit'] > 0]['realizedProfit'].mean() if win_positions > 0 else 0
        avg_loss = port_trades[port_trades['realizedProfit'] < 0]['realizedProfit'].mean() if loss_positions > 0 else 0
        
        # ROI Calculation
        total_investment = abs(port_trades['quantity']).sum()
        roi = (total_pnl / total_investment * 100) if total_investment > 0 else 0
        
        # Daily Returns for Sharpe Ratio
        daily_returns = port_trades.groupby(port_trades['time'].dt.date)['realizedProfit'].sum()
        if len(daily_returns) > 1:
            returns_mean = daily_returns.mean()
            returns_std = daily_returns.std()
            risk_free_rate = 0.02 / 252  # Assuming 2% annual risk-free rate
            sharpe_ratio = (returns_mean - risk_free_rate) / returns_std if returns_std != 0 else 0
        else:
            sharpe_ratio = 0
        
        # Maximum Drawdown
        cumulative_returns = (1 + daily_returns.pct_change()).cumprod()
        rolling_max = cumulative_returns.expanding().max()
        drawdowns = (cumulative_returns - rolling_max) / rolling_max
        mdd = abs(drawdowns.min()) if not drawdowns.empty else 0
        
        metrics.append({
            'Port_ID': port_id,
            'Total_Positions': total_positions,
            'Win_Positions': win_positions,
            'Loss_Positions': loss_positions,
            'Win_Rate': win_rate,
            'Total_PnL': total_pnl,
            'Avg_Win': avg_win,
            'Avg_Loss': avg_loss,
            'ROI': roi,
            'Sharpe_Ratio': sharpe_ratio,
            'MDD': mdd,
            'Position_Types': position_types
        })
    
    return pd.DataFrame(metrics)

In [28]:
def generate_report(metrics_df, top_20):
    """Generate detailed analysis report"""
    report = []
    report.append("Trading Performance Analysis Report")
    report.append("================================\n")
    
    report.append("Analysis Approach:")
    report.append("1. Data Processing:")
    report.append("   - Processed trade history from multiple portfolios")
    report.append("   - Classified trades by position type (long/short, open/close)")
    report.append("   - Calculated realized profits and losses\n")
    
    report.append("2. Key Metrics Calculated:")
    report.append("   - Return on Investment (ROI)")
    report.append("   - Profit and Loss (PnL)")
    report.append("   - Sharpe Ratio (Risk-adjusted returns)")
    report.append("   - Maximum Drawdown (MDD)")
    report.append("   - Win Rate and Position Analysis\n")
    
    report.append("Key Findings:")
    report.append(f"- Total Portfolios Analyzed: {len(metrics_df)}")
    report.append(f"- Average Win Rate: {metrics_df['Win_Rate'].mean():.2%}")
    report.append(f"- Average ROI: {metrics_df['ROI'].mean():.2f}%")
    report.append(f"- Best ROI: {metrics_df['ROI'].max():.2f}%")
    report.append(f"- Average Sharpe Ratio: {metrics_df['Sharpe_Ratio'].mean():.2f}\n")
    
    report.append("Top 20 Portfolios Performance Summary:")
    report.append(top_20[['Port_ID', 'ROI', 'Total_PnL', 'Win_Rate', 'Sharpe_Ratio', 'MDD']].to_string())
    
    return "\n".join(report)

In [29]:
def main():
    try:
        print("Loading and processing trades...")
        trades_df = load_and_process_trades()
        
        print("Calculating portfolio metrics...")
        metrics_df = calculate_portfolio_metrics(trades_df)
        
        # Rank portfolios
        metrics_df['final_score'] = (
            metrics_df['ROI'].rank(pct=True) * 0.3 +
            metrics_df['Sharpe_Ratio'].rank(pct=True) * 0.3 +
            metrics_df['Win_Rate'].rank(pct=True) * 0.2 +
            (1 - metrics_df['MDD'].rank(pct=True)) * 0.2
        )
        
        ranked_df = metrics_df.sort_values('final_score', ascending=False)
        top_20 = ranked_df.head(20)
        
        # Generate and save report
        report = generate_report(metrics_df, top_20)
        with open('trading_analysis_report.txt', 'w') as f:
            f.write(report)
        
        # Save detailed metrics
        top_20.to_csv('top_20_portfolios.csv', index=False)
        
        # Print top 20 summary to console
        print("\nTop 20 Portfolios Summary:")
        print(top_20[['Port_ID', 'ROI', 'Total_PnL', 'Win_Rate', 'Sharpe_Ratio', 'MDD']].to_string())
        
        # Visualizations
        plt.figure(figsize=(15, 10))
        
        # ROI vs Win Rate scatter plot
        plt.subplot(2, 2, 1)
        plt.scatter(top_20['ROI'], top_20['Win_Rate'], alpha=0.6)
        plt.title('ROI vs Win Rate (Top 20)')
        plt.xlabel('ROI (%)')
        plt.ylabel('Win Rate')
        
        # PnL Distribution
        plt.subplot(2, 2, 2)
        plt.hist(top_20['Total_PnL'], bins=10)
        plt.title('PnL Distribution (Top 20)')
        plt.xlabel('Total PnL')
        plt.ylabel('Frequency')
        
        # Sharpe Ratio vs MDD
        plt.subplot(2, 2, 3)
        plt.scatter(top_20['Sharpe_Ratio'], top_20['MDD'], alpha=0.6)
        plt.title('Sharpe Ratio vs MDD (Top 20)')
        plt.xlabel('Sharpe Ratio')
        plt.ylabel('Maximum Drawdown')
        
        # Position Types
        plt.subplot(2, 2, 4)
        win_rates = top_20['Win_Rate'].sort_values(ascending=True)
        plt.barh(range(len(win_rates)), win_rates)
        plt.yticks(range(len(win_rates)), top_20['Port_ID'][win_rates.index])
        plt.title('Win Rates by Portfolio (Top 20)')
        plt.xlabel('Win Rate')
        
        plt.tight_layout()
        plt.savefig('portfolio_analysis.png')
        plt.close()
        
        print("\nAnalysis complete. Check:")
        print("1. trading_analysis_report.txt for detailed report")
        print("2. top_20_portfolios.csv for detailed metrics")
        print("3. portfolio_analysis.png for visualizations")
        
    except Exception as e:
        print(f"Error in analysis: {str(e)}")

In [30]:
if __name__ == "__main__":
    main()

Loading and processing trades...
Processed 211277 trades from 149 portfolios
Calculating portfolio metrics...

Top 20 Portfolios Summary:
                 Port_ID        ROI     Total_PnL  Win_Rate  Sharpe_Ratio       MDD
21   3891020560590657281   1.743407   2856.300564  0.647597      1.488991  0.476400
9    4029749871687083265   3.170541   3662.136548  0.428738      1.409120  0.000000
53   4041860229502600193   1.339037   2327.336000  0.391753      1.902836  0.000000
35   3986814617275053313   0.871628  16337.461881  0.782217      1.097648  1.000000
144  4000222729738650369   0.720406   1803.620133  0.560543      0.996848  1.000000
119  3886752488982104320   0.899440   7195.178325  0.745396      0.733632  1.000000
113  3865845304835489536   0.710571   1406.066187  0.511338      0.721938  0.000000
91   3999845462424248576   0.872968   5843.152607  0.434901      1.081013  1.000000
130  3826087012661391104  12.178517    532.656974  0.583333      0.611027  1.120323
136  40236166251787998